# function_call walkthrough（细讲版）

## 目标与先修
- 目标：理解 `function_call -> 本地函数执行 -> function_call_output 回填 -> final_answer` 的完整闭环。
- 先修：了解 JSON 基本结构、Python 字典/列表、LLM tool call 基本概念。


## 流程总览
1. 让模型返回工具调用请求（包含 `call_id` 与 `arguments`）。
2. 本地解析 `arguments`，执行 `calc_portfolio_value`。
3. 构造 `function_call_output` 回填给模型。
4. 使用 `run_demo` 对照完整 trace 与最终回答。


### 环境检查（在线模块）
- 本步做什么：检查 `DEEPSEEK_API_KEY` 是否存在。
- 为什么这样做：`function_call` 依赖在线 LLM API；缺 key 时必须显式失败，避免误以为流程成功。
- 输入：环境变量 `DEEPSEEK_API_KEY`。
- 输出：通过则继续执行；失败则抛出 `RuntimeError`。
- 观察点：失败信息要明确提示“这是配置问题，不是业务逻辑问题”。


In [ ]:
import os

api_key = os.getenv("DEEPSEEK_API_KEY", "")
if not api_key:
    raise RuntimeError("缺少 DEEPSEEK_API_KEY：在线 walkthrough 不能继续执行。")

print("环境检查通过：已检测到 DEEPSEEK_API_KEY")


### 工具单元（统一 helper）
- 本步做什么：定义 `show_json` 与 `show_kv`，用于稳定展示中间对象。
- 为什么这样做：教学时必须让过程可观测，避免“函数跑完但看不到内部结构”。
- 输入：任意 Python 对象。
- 输出：可读的 JSON 字符串或键值对列表。
- 观察点：后续每步都复用这两个 helper，输出格式保持一致。


In [ ]:
import json


def show_json(obj):
    print(json.dumps(obj, ensure_ascii=False, indent=2, default=str))


def show_kv(title, mapping):
    print(title)
    for key, value in mapping.items():
        print(f"- {key}: {value}")


## 步骤拆解（逐步）

### Step 1: 导入函数并准备输入
- 本步做什么：导入 `request_model_function_call`、`calc_portfolio_value`、`run_demo` 并设置用户问题。
- 为什么这样做：先准备“同一输入”用于分步验证和端到端验证。
- 输入：一段包含持仓信息的中文指令。
- 输出：`user_text` 字符串。
- 观察点：输入里包含正负仓位，便于验证本地计算逻辑。


In [ ]:
from component import calc_portfolio_value, request_model_function_call, run_demo

user_text = "请计算组合总市值: AAPL 10@180.5, TSLA -3@210, SPY 2@500。请先调用工具再回答。"
print(user_text)


### Step 2: 请求模型 function call
- 本步做什么：调用 `request_model_function_call(user_text)`。
- 为什么这样做：先看模型是否按约定输出工具调用结构。
- 输入：`user_text`。
- 输出：包含 `call_id`、`arguments` 的字典。
- 观察点：`arguments` 必须是 JSON 字符串，后续才能被本地解析。


In [ ]:
call = request_model_function_call(user_text)
show_json(call)

call_id = str(call["call_id"])
arguments_text = str(call["arguments"])
show_kv("关键字段", {"call_id": call_id, "arguments": arguments_text})


### Step 3: 解析 arguments 并提取 positions
- 本步做什么：把 `arguments` JSON 字符串解析为 Python 对象。
- 为什么这样做：本地函数需要结构化列表输入，而不是原始字符串。
- 输入：`arguments_text`。
- 输出：`parsed_arguments` 与 `positions`。
- 观察点：确认 `positions` 是列表，且每个元素含 `symbol/qty/price`。


In [ ]:
parsed_arguments = json.loads(arguments_text)
show_json(parsed_arguments)

positions = parsed_arguments.get("positions", [])
show_json(positions)


### Step 4: 本地执行工具函数
- 本步做什么：调用 `calc_portfolio_value(positions)` 计算组合价值。
- 为什么这样做：tool call 的核心是“模型决定调用，本地执行真实计算”。
- 输入：`positions` 列表。
- 输出：本地计算结果字典。
- 观察点：检查负仓位（如空头）是否按预期影响结果。


In [ ]:
local_result = calc_portfolio_value(positions)
show_json(local_result)


### Step 5: 构造 function_call_output
- 本步做什么：组装 `{"type":"function_call_output", ...}` 回填结构。
- 为什么这样做：模型需要该结构来关联对应 `call_id` 并生成最终回答。
- 输入：`call_id` 与 `local_result`。
- 输出：`function_call_output` 字典。
- 观察点：`call_id` 必须与模型请求中的 `call_id` 完全一致。


In [ ]:
function_call_output = {
    "type": "function_call_output",
    "call_id": call_id,
    "output": json.dumps({"result": local_result}, ensure_ascii=False),
}
show_json(function_call_output)


## 端到端结果

### Step 6: 运行完整 demo 并对照 trace
- 本步做什么：调用 `run_demo(user_text)` 获取最终回答和全过程 trace。
- 为什么这样做：验证分步观察与端到端流程一致。
- 输入：`user_text`。
- 输出：`demo_result`（含 `final_answer` 与 `trace`）。
- 观察点：trace 事件顺序应覆盖“请求调用 -> 本地执行 -> 回填 -> 最终回答”。


In [ ]:
demo_result = run_demo(user_text)
show_json(demo_result)

print()
print("FINAL ANSWER:")
print(demo_result["final_answer"])


## 常见错误
- `RuntimeError: 缺少 DEEPSEEK_API_KEY`：未配置在线 key，属于环境问题。
- `JSONDecodeError`：模型返回的 `arguments` 不是合法 JSON，需要检查提示词或模型输出。
- `KeyError: call_id/arguments`：模型输出结构不完整，需检查 API 响应与解析逻辑。

## 总结
- function calling 的关键不是“模型自己算”，而是“模型发起调用，本地做真实计算，再把结果回填”。
- 分步调试时，优先看 `call_id`、`arguments`、`function_call_output` 与 `trace` 事件顺序。
